In [26]:
import re
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

training_data = pd.read_parquet('./data/train.parquet', engine='pyarrow')
test_data = pd.read_parquet('./data/test.parquet', engine='pyarrow')

print(training_data.shape)
print(test_data.shape)

(48000, 4)
(12000, 3)


In [23]:
# parse train data

target = []
text = []
test_text = []
for index, row in training_data.iterrows():
    tmp = row['Body'] + row['Title'] + row['Tags']
    text.append(tmp)
    target.append(row['target'])
    
for index, row in test_data.iterrows():
    tmp = row['Body'] + row['Title'] + row['Tags']
    test_text.append(tmp)
    
print(len(target), len(text))

48000 48000


In [24]:
# BOW

# create the transform
vectorizer = CountVectorizer(dtype=np.uint16)
# tokenize and build vocab
vectorizer.fit(text)
# encode document
bow_vector = vectorizer.transform(text)
test_vector = tfidvectorizer.transform(test_text)

# split to train and test 
train, validation, res_train, res_validation = train_test_split(bow_vector, target, test_size=.1, random_state=0)
print(train.shape, validation.shape)

# print(np.max(bow_vec), np.min(bow_vec))

(43200, 213899) (4800, 213899)


In [29]:
# KNN

k = 10
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(train, res_train)
train_prediction = knn.predict(train)
validation_prediction = knn.predict(validation)
test_prediction = knn.predict(test_vector)

print(f'k = {k}, validation_accuracy = {accuracy_score(validation_prediction, res_validation)}, train_accuracy = {accuracy_score(train_prediction, res_train)}')

k = 10, validation_accuracy = 0.5833333333333334, train_accuracy = 0.6524537037037037


In [ ]:
result = pd.DataFrame({'Id': test_data.index, 'Predicted': test_prediction})
result.to_csv('res.csv', index=False)